# New Notebook

This notebook was created automatically.

- Date created: 2025-10-20
- Environment: Windows PowerShell



In [ ]:
# Cell 1: Setup and Configuration (Corrected)

import requests
import json

# --- CONFIGURATION ---
# ⚠️ MAKE SURE YOUR LINEAR API KEY IS PASTED HERE ⚠️
API_KEY = "xxxx"

# --- Script Variables ---
# These are based on your request
TEAM_KEY = "TEA1" # From your URL: /team/TEA1/all
LABEL_NAME = "TeamA_DeepFake"
ISSUE_TITLE_PREFIX = "S2_"

# --- API Setup ---
API_URL = "https://api.linear.app/graphql"
HEADERS = {
    "Authorization": API_KEY, # <-- THIS IS THE CORRECTED LINE
    "Content-Type": "application/json"
}

# Helper function to run GraphQL queries/mutations
def run_query(query, variables=None):
    """A simple function to post a query to the Linear API."""
    payload = {'query': query}
    if variables:
        payload['variables'] = variables
    
    response = requests.post(API_URL, headers=HEADERS, json=payload)
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Query failed to run with status code {response.status_code}: {response.text}")

print("✅ Setup complete (with corrected headers). Ready to proceed.")

✅ Setup complete (with corrected headers). Ready to proceed.


In [11]:
# Cell 2: Step 1 - Find or Create the Label

# GraphQL query to find the label by name
find_label_query = """
    query($name: String!) {
      issueLabels(filter: { name: { eq: $name } }) {
        nodes {
          id
          name
        }
      }
    }
"""

# Run the query
variables = {"name": LABEL_NAME}
data = run_query(find_label_query, variables)

# Check if the label was found
labels = data['data']['issueLabels']['nodes']
label_id = None

if labels:
    label_id = labels[0]['id']
    print(f"✅ Found existing label '{LABEL_NAME}' with ID: {label_id}")
else:
    print(f"🔍 Label '{LABEL_NAME}' not found. Creating it...")
    # GraphQL mutation to create the label
    create_label_mutation = """
        mutation($name: String!) {
          issueLabelCreate(input: { name: $name }) {
            success
            issueLabel {
              id
              name
            }
          }
        }
    """
    create_data = run_query(create_label_mutation, variables)
    if create_data['data']['issueLabelCreate']['success']:
        label_id = create_data['data']['issueLabelCreate']['issueLabel']['id']
        print(f"✅ Successfully created label '{LABEL_NAME}' with ID: {label_id}")
    else:
        raise Exception("Failed to create the label.")

✅ Found existing label 'TeamA_DeepFake' with ID: 8754c942-21da-457e-b9e5-b5821e560608


In [12]:
# Cell 3: Step 2 - Find Matching Issues

# GraphQL query to get issues from a specific team
get_issues_query = """
    query($teamKey: String!) {
      team(id: $teamKey) {
        id
        issues(first: 250) { # Fetches up to 250 issues, increase if needed
          nodes {
            id
            title
            identifier
            labelIds
          }
        }
      }
    }
"""

print(f"🔎 Searching for issues starting with '{ISSUE_TITLE_PREFIX}' in team '{TEAM_KEY}'...")
variables = {"teamKey": TEAM_KEY}
data = run_query(get_issues_query, variables)

all_issues = data['data']['team']['issues']['nodes']
issues_to_update = []

for issue in all_issues:
    if issue['title'].startswith(ISSUE_TITLE_PREFIX):
        issues_to_update.append(issue)

if issues_to_update:
    print(f"✅ Found {len(issues_to_update)} matching issues:")
    for issue in issues_to_update:
        print(f"  - {issue['identifier']}: {issue['title']}")
else:
    print(f"⚠️ No issues found with the prefix '{ISSUE_TITLE_PREFIX}'. Nothing to do.")

🔎 Searching for issues starting with 'S2_' in team 'TEA1'...
✅ Found 3 matching issues:
  - TEA1-18: S2_プロジェクト計画書の作成
  - TEA1-17: S2_利用ツールを決める
  - TEA1-15: S2_目的・スコープ・マイルストンを決める


In [13]:
# Cell 4: Step 3 - Add the Label to the Issues

# GraphQL mutation to update an issue's labels
update_issue_mutation = """
    mutation($issueId: String!, $labelIds: [String!]!) {
      issueUpdate(id: $issueId, input: { labelIds: $labelIds }) {
        success
        issue {
          id
          identifier
        }
      }
    }
"""

if issues_to_update and label_id:
    print("\n🚀 Starting to update issues...")
    for issue in issues_to_update:
        # Combine existing labels with the new one, using a set to avoid duplicates
        existing_label_ids = set(issue['labelIds'])
        existing_label_ids.add(label_id)
        
        variables = {
            "issueId": issue['id'],
            "labelIds": list(existing_label_ids)
        }
        
        update_data = run_query(update_issue_mutation, variables)
        
        if update_data['data']['issueUpdate']['success']:
            print(f"  ✅ Successfully added label to {issue['identifier']}")
        else:
            print(f"  ❌ Failed to update {issue['identifier']}")
    print("\n🎉 All done!")
else:
    print("No issues were updated.")


🚀 Starting to update issues...
  ✅ Successfully added label to TEA1-18
  ✅ Successfully added label to TEA1-17
  ✅ Successfully added label to TEA1-15

🎉 All done!
